# Loop 7 Strategic Analysis

## Goal: Understand why exp_003 is the best LB and find potential improvements

## Key Questions:
1. What makes exp_003 different from other models?
2. Can we find small variations that might improve?
3. Should we submit anything or keep exp_003 as final?

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Load all candidate submissions
candidates = {}
for i in range(7):
    candidates[f'exp_{i:03d}'] = pd.read_csv(f'/home/code/submission_candidates/candidate_{i:03d}.csv')

# Load test data for analysis
test = pd.read_csv('/home/data/test.csv')
train = pd.read_csv('/home/data/train.csv')

print("Loaded all candidates and data")
print(f"Test shape: {test.shape}")
print(f"Train shape: {train.shape}")

In [ ]:
# Submission history with LB scores
submission_history = {
    'exp_000': {'cv': 0.8316, 'lb': 0.7584, 'survivors': 157, 'model': 'XGBoost 13 features'},
    'exp_001': {'cv': 0.8238, 'lb': 0.7775, 'survivors': 131, 'model': 'Simple RF 7 features'},
    'exp_003': {'cv': 0.8373, 'lb': 0.7847, 'survivors': 130, 'model': 'Threshold-Tuned Ensemble 8 features'},
    'exp_004': {'cv': 0.8373, 'lb': 0.7631, 'survivors': 131, 'model': 'Stacking'},
    'exp_005': {'cv': 0.8395, 'lb': 0.7775, 'survivors': 131, 'model': 'Feature Engineering 13 features'},
}

print("SUBMISSION HISTORY (sorted by LB)")
print("="*80)
print(f"{'Exp':<10} {'Model':<40} {'CV':<8} {'LB':<8} {'Gap':<8} {'Survivors'}")
print("-"*80)
for exp, data in sorted(submission_history.items(), key=lambda x: x[1]['lb'], reverse=True):
    gap = data['cv'] - data['lb']
    print(f"{exp:<10} {data['model']:<40} {data['cv']:.4f}  {data['lb']:.4f}  +{gap:.4f}  {data['survivors']}")

In [ ]:
# Compare predictions across all submitted models
print("\nPREDICTION COMPARISON")
print("="*80)

# Get predictions from submitted models
submitted = ['exp_000', 'exp_001', 'exp_003', 'exp_004', 'exp_005']

# Create comparison matrix
for i, exp1 in enumerate(submitted):
    for exp2 in submitted[i+1:]:
        preds1 = candidates[exp1]['Survived'].values
        preds2 = candidates[exp2]['Survived'].values
        agreement = (preds1 == preds2).sum()
        print(f"{exp1} vs {exp2}: {agreement}/418 agree ({agreement/418*100:.1f}%)")

In [ ]:
# Focus on exp_003 (best LB) - what makes it special?
print("\nEXP_003 ANALYSIS (Best LB: 0.7847)")
print("="*80)

best = candidates['exp_003']
print(f"Survivors: {best['Survived'].sum()} ({best['Survived'].mean()*100:.1f}%)")

# Compare with each other submitted model
for exp in ['exp_000', 'exp_001', 'exp_004', 'exp_005']:
    other = candidates[exp]
    diff_mask = best['Survived'].values != other['Survived'].values
    diff_count = diff_mask.sum()
    
    # Who does exp_003 predict survives that other doesn't?
    exp003_1_other_0 = ((best['Survived'].values == 1) & (other['Survived'].values == 0)).sum()
    exp003_0_other_1 = ((best['Survived'].values == 0) & (other['Survived'].values == 1)).sum()
    
    lb_diff = submission_history['exp_003']['lb'] - submission_history[exp]['lb']
    
    print(f"\nvs {exp} (LB diff: {lb_diff:+.4f}):")
    print(f"  Total differences: {diff_count}")
    print(f"  exp_003=1, {exp}=0: {exp003_1_other_0} (exp_003 predicts survive, other doesn't)")
    print(f"  exp_003=0, {exp}=1: {exp003_0_other_1} (exp_003 predicts die, other doesn't)")

In [ ]:
# Analyze the passengers where exp_003 differs from exp_001 (second best LB)
print("\nDETAILED ANALYSIS: exp_003 vs exp_001 (LB 0.7847 vs 0.7775)")
print("="*80)

best = candidates['exp_003']
rf = candidates['exp_001']

diff_mask = best['Survived'].values != rf['Survived'].values
diff_indices = np.where(diff_mask)[0]

test_diff = test.iloc[diff_indices].copy()
test_diff['exp_003_pred'] = best['Survived'].values[diff_mask]
test_diff['exp_001_pred'] = rf['Survived'].values[diff_mask]

print(f"\nTotal differences: {len(test_diff)}")
print(f"\nexp_003=1, exp_001=0 (exp_003 predicts survive, RF doesn't):")
mask = (test_diff['exp_003_pred'] == 1) & (test_diff['exp_001_pred'] == 0)
if mask.sum() > 0:
    cols = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Name']
    print(test_diff.loc[mask, cols].to_string())

print(f"\nexp_003=0, exp_001=1 (exp_003 predicts die, RF predicts survive):")
mask = (test_diff['exp_003_pred'] == 0) & (test_diff['exp_001_pred'] == 1)
if mask.sum() > 0:
    cols = ['PassengerId', 'Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare']
    print(test_diff.loc[mask, cols].to_string())

In [ ]:
# Key insight: exp_003 improved over exp_001 by +0.72% (3 passengers)
# Let's understand what exp_003 got right that exp_001 got wrong

print("\nKEY INSIGHT: exp_003 improved over exp_001 by +0.72% (3 passengers)")
print("="*80)

# The 15 differences between exp_003 and exp_001:
# - exp_003=1, exp_001=0: 7 passengers (exp_003 predicts survive, RF doesn't)
# - exp_003=0, exp_001=1: 8 passengers (exp_003 predicts die, RF predicts survive)

# Since exp_003 got 3 more correct than exp_001:
# - If exp_003 got all 7 of its "survive" predictions right, and exp_001 got 4 of its 8 "survive" predictions right
# - Then exp_003 would be +3 better

print("\nHypothesis: exp_003's Title feature helps identify:")
print("1. Young boys (Master) who survived despite being 3rd class")
print("2. Adult males (Mr) who died despite other favorable factors")
print("\nThe Title feature adds signal that pure demographic features miss.")

In [ ]:
# What about exp_006 (Simple Blending)? Why did we decide not to submit?
print("\nEXP_006 ANALYSIS (Simple Blending - NOT SUBMITTED)")
print("="*80)

blend = candidates['exp_006']
best = candidates['exp_003']

diff_mask = blend['Survived'].values != best['Survived'].values
diff_count = diff_mask.sum()

print(f"Differences from exp_003: {diff_count}")
print(f"Blend survivors: {blend['Survived'].sum()} ({blend['Survived'].mean()*100:.1f}%)")
print(f"exp_003 survivors: {best['Survived'].sum()} ({best['Survived'].mean()*100:.1f}%)")

# Analyze the 12 differing passengers
if diff_count > 0:
    diff_indices = np.where(diff_mask)[0]
    test_diff = test.iloc[diff_indices].copy()
    test_diff['exp_003_pred'] = best['Survived'].values[diff_mask]
    test_diff['blend_pred'] = blend['Survived'].values[diff_mask]
    
    print(f"\nexp_003=0, blend=1 (blend predicts survive, exp_003 doesn't):")
    mask = (test_diff['exp_003_pred'] == 0) & (test_diff['blend_pred'] == 1)
    if mask.sum() > 0:
        cols = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
        print(test_diff.loc[mask, cols].to_string())
        
    print(f"\nPattern: ALL 6 are 3rd class females traveling ALONE")
    print(f"This is the SAME pattern that caused exp_005 to fail!")

In [ ]:
# Strategic assessment: What options do we have?
print("\nSTRATEGIC ASSESSMENT")
print("="*80)

print("""
## Current Situation:
- Best LB: 0.7847 (exp_003)
- Target: 1.0 (IMPOSSIBLE - state-of-the-art is 81-85%)
- Submissions remaining: 3

## Key Learnings:
1. CV is NOT predictive of LB - Higher CV often means worse LB
2. Simpler models work better - 8 features beats 13 features
3. ~31% survival rate is optimal - 130-131 survivors
4. Alone 3rd class females are problematic - predicting survival for them is WRONG
5. Title feature helps - exp_003 uses it, exp_001 doesn't

## Options:

### Option A: Keep exp_003 as final (CONSERVATIVE)
- Best LB achieved: 0.7847
- Risk: None
- Potential: No improvement

### Option B: Try threshold variations on exp_003
- Test thresholds 0.58, 0.60, 0.62, 0.64
- Risk: Low (small changes)
- Potential: +0.5-1% if we find better threshold

### Option C: Try CatBoost with same features
- Different algorithm might generalize differently
- Risk: Medium (new algorithm)
- Potential: Unknown

### Option D: Post-process exp_003 predictions
- Force prediction to 0 for alone 3rd class females with low fares
- Risk: Low (targeted fix)
- Potential: +0.5-1% if pattern is correct

## Recommendation:
With only 3 submissions remaining and target being impossible,
focus on understanding exp_003 better and making small, targeted improvements.
""")

In [ ]:
# Let's analyze the alone 3rd class female pattern more carefully
print("\nALONE 3RD CLASS FEMALE ANALYSIS")
print("="*80)

# In training data
train_alone_3f = train[(train['Pclass'] == 3) & 
                       (train['Sex'] == 'female') & 
                       (train['SibSp'] == 0) & 
                       (train['Parch'] == 0)]

print(f"\nTraining data - Alone 3rd class females:")
print(f"  Count: {len(train_alone_3f)}")
print(f"  Survival rate: {train_alone_3f['Survived'].mean()*100:.1f}%")
print(f"  Age range: {train_alone_3f['Age'].min():.0f} - {train_alone_3f['Age'].max():.0f}")
print(f"  Fare range: ${train_alone_3f['Fare'].min():.2f} - ${train_alone_3f['Fare'].max():.2f}")

# In test data
test_alone_3f = test[(test['Pclass'] == 3) & 
                     (test['Sex'] == 'female') & 
                     (test['SibSp'] == 0) & 
                     (test['Parch'] == 0)]

print(f"\nTest data - Alone 3rd class females:")
print(f"  Count: {len(test_alone_3f)}")
print(f"  Age range: {test_alone_3f['Age'].min():.0f} - {test_alone_3f['Age'].max():.0f}")
print(f"  Fare range: ${test_alone_3f['Fare'].min():.2f} - ${test_alone_3f['Fare'].max():.2f}")

# What does exp_003 predict for these?
test_alone_3f_ids = test_alone_3f['PassengerId'].values
exp003_preds = candidates['exp_003'].set_index('PassengerId').loc[test_alone_3f_ids, 'Survived'].values

print(f"\nexp_003 predictions for alone 3rd class females:")
print(f"  Predicted survive: {exp003_preds.sum()} ({exp003_preds.mean()*100:.1f}%)")
print(f"  Predicted die: {(1-exp003_preds).sum()} ({(1-exp003_preds).mean()*100:.1f}%)")

In [ ]:
# Show the alone 3rd class females in test data with exp_003 predictions
print("\nALONE 3RD CLASS FEMALES IN TEST DATA")
print("="*80)

test_alone_3f_with_pred = test_alone_3f.copy()
test_alone_3f_with_pred['exp_003_pred'] = exp003_preds

cols = ['PassengerId', 'Age', 'Fare', 'Name', 'exp_003_pred']
print(test_alone_3f_with_pred[cols].to_string())

print(f"\nSummary:")
print(f"  Total alone 3rd class females: {len(test_alone_3f_with_pred)}")
print(f"  exp_003 predicts survive: {test_alone_3f_with_pred['exp_003_pred'].sum()}")
print(f"  exp_003 predicts die: {(1-test_alone_3f_with_pred['exp_003_pred']).sum()}")

In [ ]:
# Final strategic recommendation
print("\n" + "="*80)
print("FINAL STRATEGIC RECOMMENDATION")
print("="*80)

print("""
## Reality Check:
- Target of 1.0 (100% accuracy) is IMPOSSIBLE
- State-of-the-art for Titanic is 81-85% accuracy
- Current best LB: 0.7847 (78.47%)
- Gap to state-of-the-art: ~2-6% (8-25 passengers)

## What We've Learned:
1. exp_003 (Threshold-Tuned Ensemble, 8 features) is the best approach
2. Adding more features hurts LB (exp_005)
3. Stacking hurts LB (exp_004)
4. Simple blending predicts survival for alone 3rd class females (wrong)
5. CV is not reliable - trust LB patterns instead

## Recommendation:
Given the impossible target and limited submissions (3 remaining),
the most rational approach is:

1. **ACCEPT that we cannot beat 1.0** - It's mathematically impossible
2. **Focus on incremental improvements** - Try to get from 0.7847 to 0.79+
3. **Be conservative with submissions** - Don't waste them on risky experiments

## Next Steps:
1. Try Option D: Post-process exp_003 to force prediction=0 for alone 3rd class females
2. If that doesn't improve CV, try Option B: Threshold variations
3. Save at least 1 submission for final attempt

## Expected Outcome:
- Best case: 0.79-0.80 LB (1-2% improvement)
- Realistic: 0.7847 remains best (no improvement)
- Worst case: Regression if we submit wrong experiments
""")

In [ ]:
# Check what the training survival rate is for alone 3rd class females by age
print("\nALONE 3RD CLASS FEMALE SURVIVAL BY AGE (Training Data)")
print("="*80)

train_alone_3f = train[(train['Pclass'] == 3) & 
                       (train['Sex'] == 'female') & 
                       (train['SibSp'] == 0) & 
                       (train['Parch'] == 0)].copy()

# Create age bins
train_alone_3f['AgeBin'] = pd.cut(train_alone_3f['Age'], bins=[0, 18, 30, 45, 100], labels=['0-18', '18-30', '30-45', '45+'])

print(train_alone_3f.groupby('AgeBin')['Survived'].agg(['count', 'sum', 'mean']).round(3))

print(f"\nOverall survival rate: {train_alone_3f['Survived'].mean()*100:.1f}%")
print(f"\nKey insight: Alone 3rd class females have ~50% survival rate in training,")
print(f"but exp_005 and exp_006 failed by predicting survival for them.")
print(f"This suggests test set has LOWER survival rate for this group.")

In [ ]:
# Check what exp_003 predicts for alone 3rd class females by age
print("\nEXP_003 PREDICTIONS FOR ALONE 3RD CLASS FEMALES BY AGE")
print("="*80)

test_alone_3f = test[(test['Pclass'] == 3) & 
                     (test['Sex'] == 'female') & 
                     (test['SibSp'] == 0) & 
                     (test['Parch'] == 0)].copy()

test_alone_3f_ids = test_alone_3f['PassengerId'].values
test_alone_3f['exp_003_pred'] = candidates['exp_003'].set_index('PassengerId').loc[test_alone_3f_ids, 'Survived'].values

# Create age bins
test_alone_3f['AgeBin'] = pd.cut(test_alone_3f['Age'], bins=[0, 18, 30, 45, 100], labels=['0-18', '18-30', '30-45', '45+'])

print(test_alone_3f.groupby('AgeBin')['exp_003_pred'].agg(['count', 'sum', 'mean']).round(3))

print(f"\nexp_003 predicts {test_alone_3f['exp_003_pred'].sum()}/{len(test_alone_3f)} alone 3rd class females survive")
print(f"({test_alone_3f['exp_003_pred'].mean()*100:.1f}%)")

In [ ]:
# What if we force all alone 3rd class females to die?
print("\nWHAT IF WE FORCE ALL ALONE 3RD CLASS FEMALES TO DIE?")
print("="*80)

# Create modified exp_003 predictions
exp003_modified = candidates['exp_003'].copy()

# Get IDs of alone 3rd class females
test_alone_3f_ids = test[(test['Pclass'] == 3) & 
                         (test['Sex'] == 'female') & 
                         (test['SibSp'] == 0) & 
                         (test['Parch'] == 0)]['PassengerId'].values

# How many would change?
original_preds = exp003_modified.set_index('PassengerId').loc[test_alone_3f_ids, 'Survived'].values
changes = original_preds.sum()

print(f"Alone 3rd class females in test: {len(test_alone_3f_ids)}")
print(f"exp_003 predicts survive: {changes}")
print(f"If we force all to die: {changes} predictions would change")

# New survival count
new_survivors = exp003_modified['Survived'].sum() - changes
print(f"\nOriginal survivors: {exp003_modified['Survived'].sum()} ({exp003_modified['Survived'].mean()*100:.1f}%)")
print(f"New survivors: {new_survivors} ({new_survivors/len(exp003_modified)*100:.1f}%)")

print(f"\nRISK: This is a drastic change. We're assuming ALL alone 3rd class females died.")
print(f"The training data shows ~50% survival rate for this group.")
print(f"This might be too aggressive.")

In [ ]:
# More targeted approach: Force prediction=0 for alone 3rd class females with LOW fares
print("\nTARGETED APPROACH: Force prediction=0 for alone 3rd class females with LOW fares")
print("="*80)

# In training data, what's the survival rate by fare for alone 3rd class females?
train_alone_3f = train[(train['Pclass'] == 3) & 
                       (train['Sex'] == 'female') & 
                       (train['SibSp'] == 0) & 
                       (train['Parch'] == 0)].copy()

train_alone_3f['FareBin'] = pd.cut(train_alone_3f['Fare'], bins=[0, 8, 15, 100], labels=['Low (<$8)', 'Medium ($8-15)', 'High (>$15)'])

print("Training data - Alone 3rd class female survival by fare:")
print(train_alone_3f.groupby('FareBin')['Survived'].agg(['count', 'sum', 'mean']).round(3))

# In test data
test_alone_3f = test[(test['Pclass'] == 3) & 
                     (test['Sex'] == 'female') & 
                     (test['SibSp'] == 0) & 
                     (test['Parch'] == 0)].copy()

test_alone_3f['FareBin'] = pd.cut(test_alone_3f['Fare'], bins=[0, 8, 15, 100], labels=['Low (<$8)', 'Medium ($8-15)', 'High (>$15)'])
test_alone_3f_ids = test_alone_3f['PassengerId'].values
test_alone_3f['exp_003_pred'] = candidates['exp_003'].set_index('PassengerId').loc[test_alone_3f_ids, 'Survived'].values

print("\nTest data - exp_003 predictions for alone 3rd class females by fare:")
print(test_alone_3f.groupby('FareBin')['exp_003_pred'].agg(['count', 'sum', 'mean']).round(3))

In [ ]:
# Identify the specific passengers we might want to change
print("\nSPECIFIC PASSENGERS TO CONSIDER CHANGING")
print("="*80)

# Alone 3rd class females with low fares that exp_003 predicts survive
test_alone_3f_low_fare = test_alone_3f[(test_alone_3f['Fare'] < 8) & (test_alone_3f['exp_003_pred'] == 1)]

print(f"\nAlone 3rd class females with Fare < $8 that exp_003 predicts survive:")
if len(test_alone_3f_low_fare) > 0:
    cols = ['PassengerId', 'Age', 'Fare', 'Name']
    print(test_alone_3f_low_fare[cols].to_string())
else:
    print("  None - exp_003 already predicts all low-fare alone 3rd class females die")

# What about medium fare?
test_alone_3f_med_fare = test_alone_3f[(test_alone_3f['Fare'] >= 8) & (test_alone_3f['Fare'] < 15) & (test_alone_3f['exp_003_pred'] == 1)]

print(f"\nAlone 3rd class females with Fare $8-15 that exp_003 predicts survive:")
if len(test_alone_3f_med_fare) > 0:
    cols = ['PassengerId', 'Age', 'Fare', 'Name']
    print(test_alone_3f_med_fare[cols].to_string())
else:
    print("  None")

In [ ]:
# Summary of findings
print("\n" + "="*80)
print("SUMMARY OF FINDINGS")
print("="*80)

print("""
## Key Finding:
exp_003 already predicts DEATH for most alone 3rd class females!
- Only 2 alone 3rd class females are predicted to survive by exp_003
- Both have medium fares ($8-15)

## Why exp_006 (Simple Blending) failed:
- The blend predicted survival for 6 alone 3rd class females
- This is 4 MORE than exp_003 predicts
- These 4 extra "survive" predictions are likely WRONG

## Conclusion:
exp_003 is already well-calibrated for the alone 3rd class female pattern.
Trying to "fix" this pattern would likely make things worse, not better.

## Strategic Recommendation:
1. exp_003 is the best model we have
2. Further modifications are unlikely to improve LB
3. With 3 submissions remaining, be VERY conservative
4. Consider keeping exp_003 as final submission
""")

In [ ]:
# Final check: What's the realistic best we can achieve?
print("\nREALISTIC ASSESSMENT")
print("="*80)

print("""
## Target: 1.0 (100% accuracy)
- This is IMPOSSIBLE for Titanic
- State-of-the-art is 81-85% accuracy
- Even the best Kaggle solutions don't achieve 100%

## Current Best: 0.7847 (78.47%)
- This is competitive (top ~20-30% on leaderboard)
- Gap to state-of-the-art: ~2-6%

## What would it take to improve?
- Need to correctly predict ~2-6 more passengers
- But every change risks making other predictions wrong
- CV is not reliable for predicting LB improvement

## Recommendation:
Given the impossible target and limited submissions,
the most rational approach is to ACCEPT that we cannot beat 1.0
and focus on preserving our best LB score (0.7847).

DO NOT submit exp_006 (Simple Blending) - it will likely regress.
Consider keeping exp_003 as the final submission.
""")